<a href="https://colab.research.google.com/github/microsoft/autogen/blob/main/notebook/agentchat_groupchat_research.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Auto Generated Agent Chat: Performs Research with Multi-Agent Group Chat

AutoGen offers conversable agents powered by LLM, tool, or human, which can be used to perform tasks collectively via automated chat. This framework allows tool use and human participation through multi-agent conversation.
Please find documentation about this feature [here](https://microsoft.github.io/autogen/docs/Use-Cases/agent_chat).

## Requirements

AutoGen requires `Python>=3.8`. To run this notebook example, please install:
```bash
pip install pyautogen
```

In [1]:
%%capture --no-stderr
%pip install pyautogen~=0.2.0b4

## Set your API Endpoint

The [`config_list_from_json`](https://microsoft.github.io/autogen/docs/reference/oai/openai_utils#config_list_from_json) function loads a list of configurations from an environment variable or a json file.

In [3]:
import autogen

config_list_mistral = [
    {
        'base_url': "https://minolta-heading-pal-travelers.trycloudflare.com/openai",
        'api_key': "NULL"
    }
]

It first looks for environment variable "OAI_CONFIG_LIST" which needs to be a valid json string. If that variable is not found, it then looks for a json file named "OAI_CONFIG_LIST". It filters the configs by models (you can filter by other keys as well).

The config list looks like the following:
```python
config_list = [
    {
        'model': 'gpt-4-32k',
        'api_key': '<your OpenAI API key here>',
    },
    {
        'model': 'gpt-4-32k',
        'api_key': '<your Azure OpenAI API key here>',
        'base_url': '<your Azure OpenAI API base here>',
        'api_type': 'azure',
        'api_version': '2023-06-01-preview',
    },
    {
        'model': 'gpt-4-32k-0314',
        'api_key': '<your Azure OpenAI API key here>',
        'base_url': '<your Azure OpenAI API base here>',
        'api_type': 'azure',
        'api_version': '2023-06-01-preview',
    },
]
```

You can set the value of config_list in any way you prefer. Please refer to this [notebook](https://github.com/microsoft/autogen/blob/main/notebook/oai_openai_utils.ipynb) for full code examples of the different methods.

## Construct Agents

In [4]:
gpt4_config = {
    "cache_seed": 42,  # change the cache_seed for different trials
    "temperature": 0,
    "config_list": config_list_mistral,
    "timeout": 120,
}
user_proxy = autogen.UserProxyAgent(
   name="Admin",
   system_message="A human admin. Interact with the planner to discuss the plan. Plan execution needs to be approved by this admin.",
   code_execution_config=False,
)
engineer = autogen.AssistantAgent(
    name="Engineer",
    llm_config=gpt4_config,
    system_message='''Engineer. You follow an approved plan. You write python/shell code to solve tasks. Wrap the code in a code block that specifies the script type. The user can't modify your code. So do not suggest incomplete code which requires others to modify. Don't use a code block if it's not intended to be executed by the executor.
Don't include multiple code blocks in one response. Do not ask others to copy and paste the result. Check the execution result returned by the executor.
If the result indicates there is an error, fix the error and output the code again. Suggest the full code instead of partial code or code changes. If the error can't be fixed or if the task is not solved even after the code is executed successfully, analyze the problem, revisit your assumption, collect additional info you need, and think of a different approach to try.
''',
)
scientist = autogen.AssistantAgent(
    name="Scientist",
    llm_config=gpt4_config,
    system_message="""Scientist. You follow an approved plan. You are able to categorize papers after seeing their abstracts printed. You don't write code."""
)
planner = autogen.AssistantAgent(
    name="Planner",
    system_message='''Planner. Suggest a plan. Revise the plan based on feedback from admin and critic, until admin approval.
The plan may involve an engineer who can write code and a scientist who doesn't write code.
Explain the plan first. Be clear which step is performed by an engineer, and which step is performed by a scientist.
''',
    llm_config=gpt4_config,
)
executor = autogen.UserProxyAgent(
    name="Executor",
    system_message="Executor. Execute the code written by the engineer and report the result.",
    human_input_mode="NEVER",
    code_execution_config={"last_n_messages": 3, "work_dir": "paper"},
)
critic = autogen.AssistantAgent(
    name="Critic",
    system_message="Critic. Double check plan, claims, code from other agents and provide feedback. Check whether the plan includes adding verifiable info such as source URL.",
    llm_config=gpt4_config,
)
groupchat = autogen.GroupChat(agents=[user_proxy, engineer, scientist, planner, executor, critic], messages=[], max_round=50)
manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=gpt4_config)

## Start Chat

In [5]:
user_proxy.initiate_chat(
    manager,
    message="""
find papers on LLM applications from arxiv in the last week, create a markdown table of different domains.
""",
)

Admin (to chat_manager):


find papers on LLM applications from arxiv in the last week, create a markdown table of different domains.


--------------------------------------------------------------------------------


 Planner. Here's the plan:
1. I, as the Planner, will create a Python script to query Arxiv for papers related to LLM applications in the last week. The Engineer can write this code if needed.
2. Once we have the script, the Executor will run it and provide us with the results.
3. The Scientist will then categorize these papers into different domains based on their abstracts.
4. We will create a markdown table to display the number of papers in each domain.
5. I will ensure that this plan includes verifiable information such as the URL for the Arxiv API or documentation we'll be using.

Role: Planner


Engineer (to chat_manager):

 ```python
import requests
import json
import time

# Set up API key and query parameters
api_key = "your_api_key"
query = "LLM applications"
start_date = int(time.time() - 7 * 24 * 60 * 60)  # Last week in Unix time format

# Set up base URL and headers for API requests
base_url = "https://api.arxiv.org/query"
headers = {"apikey": api_key}

# Define function to get papers from ArXiv API
def get_papers(query, start_date=None):
    params = {
        "search_query": query,
        "sort_by": "relevance",
        "start": 0,
        "max_results": 100,
        "from": start_date
    }
    response = requests.get(base_url, params=params, headers=headers)
    return response.json()["records"]

# Get papers from ArXiv API for the given query and date range
papers = get_papers(query, start_date)

# Create markdown table of different domains
print("# LLM Applications in Different Domains (Last Week)")
print("| Domain | Number of Papers |")
print("|--------|-------

 Engineer. I will write the Python code to find papers on LLM applications from ArXiv in the last week and create a markdown table of different domains. The Scientist role will not be involved in this task as it does not require categorizing papers after seeing their abstracts, but only analyzing them based on their given labels. Here's the Python code for your reference:

```python
import requests
import json
import time

# Set up API key and query parameters
api_key = "your_api_key"
query = "LLM applications"
start_date = int(time.time() - 7 * 24 * 60 * 60)  # Last week in Unix time format

# Set up base URL and headers for API requests
base_url = "https://api.arxiv.org/query"
headers = {"apikey": api_key}

# Define function to get papers from ArXiv API
def get_papers(query, start_date=None):
    params = {
        "search_query": query,
        "sort_by": "relevance",
        "start": 0,
        "max_results": 100,
        "from": start_date
    }
    response = requests.get(base_ur

Scientist (to chat_manager):

 I cannot directly run or test this code as it is written in Python and requires an ArXiv API key, which I don't have. However, I can explain what the code does and help you understand how to use it.

This script uses the ArXiv API to search for papers on LLM applications (likely referring to Large Language Models) published in the last week. It then creates a markdown table displaying the number of papers found in different domains, including Machine Learning, Natural Language Processing, Computer Vision, and Robotics.

Here's a brief explanation of the code:

1. The script sets up variables for the API key, query (LLM applications), start date (last week), base URL, and headers for API requests.
2. A function `get_papers()` is defined to get papers from ArXiv API with the given query, sorting by relevance, setting the maximum number of results to 100, and starting from the specified date.
3. The function `get_papers()` makes an HTTP GET request to the Ar

 Planner. I suggest the following plan:

1. As a Scientist, you will help categorize the papers based on their abstracts into different domains such as Machine Learning, Natural Language Processing, Computer Vision, and Robotics. This will be done manually once we have the list of papers from the API.
2. An Engineer will write Python code to interact with the ArXiv API to retrieve papers related to LLM applications published in the last week. The output from the API should be a list of papers, which includes their titles, abstracts, and domain categories.
3. Once the Engineer has provided the list of papers, I will write Python code as a Planner that extracts the domains from the paper information using the Scientist's categorization. The script will then create a markdown table displaying the number of papers found in each domain.
4. The Executor will test the Engineer and Planner's parts of the plan separately, making sure both parts work correctly before combining them to get the co

Planner (to chat_manager):

 The plan is to write a Python script that fetches papers related to LLM applications (Large Language Models) from ArXiv in the last week and generates a markdown table showing the number of papers in different domains, such as Machine Learning, Natural Language Processing, Computer Vision, and Robotics.

In this plan, an engineer is responsible for writing the Python code to interact with the ArXiv API, fetch the required data, create and format the markdown table. They will also ensure that the script runs correctly and efficiently.

A scientist, who doesn't write code but has a strong understanding of the domain (in this case, machine learning and related fields), can provide valuable feedback on the accuracy of the domains selected for categorizing the papers, improve the list of domains if needed, or suggest any domain-specific enhancements to make the script more effective in extracting relevant information.

Here's a suggested plan:

1. Engineer write

KeyboardInterrupt: ignored

## Create Group Chat without Critic for Comparison

In [6]:
groupchat_nocritic = autogen.GroupChat(agents=[user_proxy, engineer, scientist, planner, executor], messages=[], max_round=50)
for agent in groupchat.agents:
    agent.reset()
manager_nocritic = autogen.GroupChatManager(groupchat=groupchat_nocritic, llm_config=gpt4_config)
user_proxy.initiate_chat(
    manager_nocritic,
    message="""
find papers on LLM applications from arxiv in the last week, create a markdown table of different domains.
""",
)

Admin (to chat_manager):


find papers on LLM applications from arxiv in the last week, create a markdown table of different domains.


--------------------------------------------------------------------------------
Provide feedback to chat_manager. Press enter to skip and use auto-reply, or type 'exit' to end the conversation: exit
